In [ ]:
%matplotlib inline

In [ ]:
n = 7
img_number = tag.iloc[n,0]
img_tag = tag.iloc[n,1]

print('Image number: {}'.format(img_number))
print('Image tag: {}'.format(img_tag))

In [ ]:
def show_img_tags(image, tag):
    """Show image with tags"""
    plt.imshow(image)
    plt.title(tag)
    plt.pause(0.001)

plt.figure()
show_img_tags(io.imread(os.path.join('../extra_data/images/', str(img_number)+'.jpg')),
             img_tag)
plt.show()

In [ ]:
anime_tag_dataset = anime_tag_Dataset(csv_file='../extra_data/tags.csv',
                                     root_dir='../extra_data/images/')

In [2]:
from __future__ import print_function
import torch 
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
import os 
import numpy as np
import pandas as pd 
from skimage import io, transform
import matplotlib.pyplot as plt

In [3]:
tag = pd.read_csv('../extra_data/tags.csv', header=None)

In [4]:
context_size = 2
embedding_dim = 30
raw_text = ['orange hair', 'white hair', 'aqua hair', 'gray hair',
'green hair', 'red hair', 'purple hair', 'pink hair',
'blue hair', 'black hair', 'brown hair', 'blonde hair',
'gray eyes', 'black eyes', 'orange eyes',
'pink eyes', 'yellow eyes', 'aqua eyes', 'purple eyes',
'green eyes', 'brown eyes', 'red eyes', 'blue eyes']

vocab = set()
for i in raw_text:
    vocab.update(i.split())
word_to_idx = {word: float(i) for i, word in enumerate(vocab)}

In [6]:
class anime_tag_Dataset(Dataset):
    """Anime tag dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with tags.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.tag_csv = pd.read_csv(csv_file, header=None)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.tag_csv)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,
                                str(self.tag_csv.iloc[idx, 0])+'.jpg')
        image = io.imread(img_name)
        tags = self.tag_csv.iloc[idx, 1]
        sample = {'image': image, 'tags': tags}
        
        if self.transform:
            sample = self.transform(sample)

        return sample

In [7]:
BATCH_SIZE = 1
IMAGE_SIZE = 64

class Normalize(object):
    """Nornalize images"""
    def __call__(self, sample):
        image, tags = sample['image'], sample['tags']
        image = transforms.functional.normalize(image,
                                mean=(0.5, 0.5, 0.5),
                                std=(0.5, 0.5, 0.5))
        return {'image': torch.from_numpy(image),
                'tags': torch.from_numpy(tags)}

class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, tags = sample['image'], sample['tags']
        tags = [word_to_idx[w] for w in tags.split()]
        tags = np.array(tags)
        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image.transpose((2, 0, 1))
        return {'image': torch.from_numpy(image),
                'tags': torch.from_numpy(tags)}

In [8]:
transformed_dataset = anime_tag_Dataset(csv_file='../extra_data/tags.csv',
                                     root_dir='../extra_data/images/',
                                    transform=transforms.Compose([
                                        ToTensor(),
                                    ]))

In [9]:
dataloader = DataLoader(transformed_dataset, batch_size=BATCH_SIZE,
                       shuffle=True, num_workers=8)

In [ ]:
#for i_batch, sample_batched in enumerate(dataloader):
#    print(i_batch, sample_batched['image'], sample_batched['tags'])

In [10]:
class Generator(nn.Module):

    def __init__(self, input_size=100, label_size=4, num_classes=3*64*64):
        super(Generator, self).__init__()
        self.layer = nn.Sequential(
                    nn.Linear(input_size+label_size, 256),
                    nn.LeakyReLU(0.2),
                    nn.Linear(256,256),
                    nn.LeakyReLU(0.2),
                    nn.Linear(256, num_classes),
                    nn.Tanh()
        )
    def forward(self, image, condition):
        x, y = image.view(image.size(0), -1), condition.view(condition.size(0), -1).float()
        v = torch.cat((x,y), 1)
        output = self.layer(v)
        output = output.view(x.size(0), 3, 64, 64)
        return output

In [18]:
class  Discriminator(nn.Module):
    """
        Simple Discriminator w/ MLP
    """
    def __init__(self, input_size=64*64*3, label_size=4, num_classes=1):
        super(Discriminator, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Linear(input_size+4, 256),
            nn.ReLU(),
            nn.Dropout(),
        )
        self.layer2 = nn.Sequential(
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Dropout(),
        )
        self.layer3 = nn.Sequential(
            nn.Linear(256, num_classes),
            nn.Sigmoid(),
        )
    
    def forward(self, x, y):        
        x = x.view(x.size(0), -1)
        v = torch.cat([x,y.type_as(x)], 1) # v: [input, label] concatenated vector
        y_ = self.layer1(v)
        y_ = self.layer2(y_)
        y_ = self.layer3(y_)
        return y_

In [19]:
MODEL_NAME = 'ConditionalGAN'

In [20]:
D = Discriminator()
G = Generator()

In [21]:
criterion = nn.BCELoss()
D_opt = torch.optim.Adam(D.parameters())
G_opt = torch.optim.Adam(G.parameters())

In [22]:
max_epoch  = 100 # need more than 200 epochs for training generator
step = 0
n_critic = 5 # for training more k steps about Discriminator
n_noise = 100

D_labels = torch.ones(BATCH_SIZE)#.cuda() # Discriminator Label to real
D_fakes = torch.zeros(BATCH_SIZE)#.cuda() # Discriminator Label to fake

In [32]:
for epoch in range(max_epoch):
    for idx, sample in enumerate(dataloader):
        step += 1 
        # Training Discriminator
        x = Variable(sample['image'].float(), requires_grad = False)
        y = Variable(sample['tags'].float(), requires_grad = False)
        print(x)
        print(y)
        x_outputs = D(x, y)
        D_x_loss = criterion(x_outputs, D_labels)

        z = to_cuda(torch.randn(BATCH_SIZE, n_noise))
        z_outputs = D(G(z, y), y)
        D_z_loss = criterion(z_outputs, D_fakes)
        D_loss = D_x_loss + D_z_loss
        
        D.zero_grad()
        D_loss.backward()
        D_opt.step()
        
        if step % n_critic == 0:
            # Training Generator
            z = torch.randn(BATCH_SIZE, n_noise)#.cuda()
            z_outputs = D(G(z, y), y)
            G_loss = criterion(z_outputs, D_labels)

            G.zero_grad()
            G_loss.backward()
            G_opt.step()
        
        if step % 1000 == 0:
            print('Epoch: {}/{}, Step: {}, D Loss: {}, G Loss: {}'.format(epoch, max_epoch, step, D_loss.data[0], G_loss.data[0]))
            
        if epoch % 5 == 0:
            G.eval()
            img = get_sample_image(G)
            scipy.misc.imsave('sample/{}_epoch_{}_type1.jpg'.format(MODEL_NAME, epoch), img)
            G.train()

Variable containing:
(0 ,0 ,.,.) = 
  253  255  253  ...   250  245  246
  254  253  255  ...   252  248  247
  255  255  249  ...   250  247  245
      ...         ⋱        ...      
   34   28   29  ...   247  249  247
   30   28   31  ...   248  248  248
   31   30   37  ...   248  247  249

(0 ,1 ,.,.) = 
  249  254  251  ...   249  243  246
  250  249  254  ...   251  246  247
  253  253  247  ...   249  245  245
      ...         ⋱        ...      
   38   32   31  ...   247  248  243
   34   32   35  ...   248  247  244
   35   34   41  ...   248  246  245

(0 ,2 ,.,.) = 
  248  253  252  ...   247  244  248
  249  250  255  ...   249  247  249
  254  254  250  ...   247  246  247
      ...         ⋱        ...      
   50   44   44  ...   247  246  242
   46   44   47  ...   248  245  243
   47   46   53  ...   248  244  244
[torch.FloatTensor of size 1x3x64x64]

Variable containing:
 5  8  5  3
[torch.FloatTensor of size 1x4]



AttributeError: 'torch.FloatTensor' object has no attribute 'requires_grad'

In [ ]:
for epoch in range(max_epoch):
    for idx, (images, labels) in enumerate(dataloader):
        print(idx, images, labels)

In [25]:
print(torch.__version__)

0.3.1.post2
